## Titanic with Keras

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame

import keras
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Flatten, Dense, Activation, Dropout

Using TensorFlow backend.


## setup

In [2]:
DATA_HOME_DIR = "/home/tsu-nera/repo/kaggle/input/titanic/"
row_data = pd.read_csv(DATA_HOME_DIR + 'train.csv', index_col=0)
test_data = pd.read_csv(DATA_HOME_DIR + 'test.csv', index_col=0)

In [3]:
row_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 11 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Name        891 non-null object
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Ticket      891 non-null object
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 83.5+ KB


In [4]:
test_ind = test_data.index

data= row_data[['Survived','Pclass','Sex','Age','SibSp','Parch','Cabin']]
tdata = test_data[['Pclass','Sex','Age','SibSp','Parch', 'Cabin']]

In [5]:
## クラスごとに分割
Pclass = pd.get_dummies(data['Pclass'])
Pclass.columns=['1st','2nd','3rd']
Pclass = Pclass.drop('1st',axis=1)
tPclass = pd.get_dummies(tdata['Pclass'])
tPclass.columns=['1st','2nd','3rd']
tPclass = tPclass.drop('1st',axis=1)

In [6]:
## 女性、男性、子供ごとに分割
Sex = pd.get_dummies(data['Sex']).drop('male',axis=1)
tSex = pd.get_dummies(tdata['Sex']).drop('male',axis=1)

def male_female_child(passenger):
    age,sex = passenger
    if np.isnan(age):
        age = 30
    if age < 16:
        return 'child'
    else:
        return sex

data['person'] = data[['Age','Sex']].apply(male_female_child,axis=1)
Age_cat = pd.get_dummies(data['person']).drop('child',axis=1)
tdata['person'] = tdata[['Age','Sex']].apply(male_female_child,axis=1)
tAge_cat = pd.get_dummies(tdata['person']).drop('child',axis=1)

/home/tsu-nera/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel/__main__.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/tsu-nera/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [7]:
# 独身かそうでないかで分類
data['Alone'] = data.Parch + data.SibSp
data['Alone'].loc[data['Alone'] >0] = 0
data['Alone'].loc[data['Alone'] == 0] = 1

tdata['Alone'] = tdata.Parch + tdata.SibSp
tdata['Alone'].loc[tdata['Alone'] >0] = 0
tdata['Alone'].loc[tdata['Alone'] == 0] = 1

/home/tsu-nera/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/tsu-nera/anaconda3/envs/kaggle/lib/python3.6/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/tsu-nera/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/

In [8]:
def get_level(deck):
    if pd.isnull(deck):
        deck = 'CXX'
    return deck[0]

data['Level'] = data.Cabin.apply(get_level)
tdata['Level'] = tdata.Cabin.apply(get_level)

data = data[data.Level != 'T']
Level = pd.get_dummies(data['Level']).drop('C',axis=1)
tLevel = pd.get_dummies(tdata['Level']).drop('C',axis=1)

/home/tsu-nera/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/tsu-nera/anaconda3/envs/kaggle/lib/python3.6/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,A,B,D,E,F,G
PassengerId,,,,,,
1,0,0,0,0,0,0
2,0,0,0,0,0,0
3,0,0,0,0,0,0
4,0,0,0,0,0,0
5,0,0,0,0,0,0
6,0,0,0,0,0,0
7,0,0,0,1,0,0
8,0,0,0,0,0,0
9,0,0,0,0,0,0


In [17]:
Data_tmp = data[['Survived', 'Alone']]

Merge_data = pd.merge(Data_tmp,Pclass,right_index=True,left_index=True)
Merge_data = pd.merge(Merge_data,Sex,right_index=True,left_index=True)
Merge_data = pd.merge(Merge_data,Age_cat,right_index=True,left_index=True)
Merge_data = pd.merge(Merge_data,Level,right_index=True,left_index=True)
y = Merge_data['Survived'].values
x = Merge_data.drop('Survived',axis=1).values

Data_tmp = tdata[['Alone']]

Merge_data = pd.merge(Data_tmp,tPclass,right_index=True,left_index=True)
Merge_data = pd.merge(Merge_data,tSex,right_index=True,left_index=True)
Merge_data = pd.merge(Merge_data,tAge_cat,right_index=True,left_index=True)
Merge_data = pd.merge(Merge_data,tLevel,right_index=True,left_index=True)
tx = Merge_data

(x.shape, y.shape, tx.shape)
tx

,Alone,2nd,3rd,female_x,female_y,male,A,B,D,E,F,G
PassengerId,,,,,,,,,,,,
892,1,0,1,0,0,1,0,0,0,0,0,0
893,1,0,1,1,1,0,0,0,0,0,0,0
894,1,1,0,0,0,1,0,0,0,0,0,0
895,1,0,1,0,0,1,0,0,0,0,0,0
896,1,0,1,1,1,0,0,0,0,0,0,0
897,1,0,1,0,0,0,0,0,0,0,0,0
898,1,0,1,1,1,0,0,0,0,0,0,0
899,1,1,0,0,0,1,0,0,0,0,0,0
900,1,0,1,1,1,0,0,0,0,0,0,0


## Build Model

In [11]:
# create model
model = Sequential()
model.add(Dense(64, input_shape=(12,)))
model.add(BatchNormalization())
model.add(Activation('relu'))

for i in range(0,7):
    model.add(Dense(units=64))
    model.add(BatchNormalization())
    model.add(Activation('relu'))

model.add(Dropout(0.5))
model.add(Dense(units=1))
model.add(Activation('linear'))

model.compile(loss='mean_squared_error', optimizer='rmsprop')

## Training

In [12]:
model.fit(x, y, epochs=400)

Epoch 1/400
890/890 [==============================] - 2s - loss: 1.7559     
Epoch 2/400
890/890 [==============================] - 0s - loss: 1.1116     
Epoch 3/400
890/890 [==============================] - 0s - loss: 0.7577     
Epoch 4/400
890/890 [==============================] - 0s - loss: 0.7327     
Epoch 5/400
890/890 [==============================] - 0s - loss: 0.6765     
Epoch 6/400
890/890 [==============================] - 0s - loss: 0.5693     
Epoch 7/400
890/890 [==============================] - 0s - loss: 0.4861     
Epoch 8/400
890/890 [==============================] - 0s - loss: 0.4285     
Epoch 9/400
890/890 [==============================] - 0s - loss: 0.3530     
Epoch 10/400
890/890 [==============================] - 0s - loss: 0.3028     
Epoch 11/400
890/890 [==============================] - 0s - loss: 0.2436     
Epoch 12/400
890/890 [==============================] - 0s - loss: 0.2274     
Epoch 13/400
890/890 [==============================] - 0s - 

890/890 [==============================] - 0s - loss: 0.1346     
Epoch 207/400
890/890 [==============================] - 0s - loss: 0.1359     
Epoch 208/400
890/890 [==============================] - 0s - loss: 0.1350     
Epoch 209/400
890/890 [==============================] - 0s - loss: 0.1359     
Epoch 210/400
890/890 [==============================] - 0s - loss: 0.1398     
Epoch 211/400
890/890 [==============================] - 0s - loss: 0.1328     
Epoch 212/400
890/890 [==============================] - 0s - loss: 0.1349     
Epoch 213/400
890/890 [==============================] - 0s - loss: 0.1345     
Epoch 214/400
890/890 [==============================] - 0s - loss: 0.1379     
Epoch 215/400
890/890 [==============================] - 0s - loss: 0.1344     
Epoch 216/400
890/890 [==============================] - 0s - loss: 0.1368     
Epoch 217/400
890/890 [==============================] - 0s - loss: 0.1362     
Epoch 218/400
890/890 [==============================]

## Testing

In [13]:
p_survived = model.predict_classes(tx.values)

 32/418 [=>............................] - ETA: 1s

In [14]:
submission = pd.DataFrame()
submission['PassengerId'] = test_ind
submission['Survived'] = p_survived

In [15]:
submission.to_csv('submission.csv', index=False)